In [3]:
import pickle
import os

# For download embeddings model
from huggingface_hub import snapshot_download

# For embeddings and vector stores
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

%load_ext autoreload
%autoreload 2

# Settings

In [187]:
embeddings_model_name = "intfloat/multilingual-e5-large-instruct"
embeddings_path = f"./embedding_model/{embeddings_model_name}"

# Extract txts from Website


In this notebook, I will show you how to extract text from a website using the BeautifulSoup library in Python.  
BeautifulSoup is a Python library that is used to extract data from HTML and XML files. It provides a simple way to navigate the HTML document and extract the data you need.  
From 'https://home.cern/science', I extract the links about the science topics and insert them into a list `urls`.

In [ ]:
import bs4
import requests

search_urls = ['https://home.cern',
                'https://home.cern/science/physics',
                'https://home.cern/science/accelerators',
                'https://home.cern/science/engineering',
                'https://home.cern/science/experiments',
                'https://home.cern/science/computing'
                ]
urls = []
for url in search_urls:
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a')
    links = [link.get('href') for link in links if link.get('href') is not None]
    for l in links:
        if l.startswith('http') and 'science' in l:
            urls.append(l)
        elif l not in urls and 'science' in l:
            urls.append('https://home.cern' + l)
        
tmp = []
for _url in urls:
    if _url not in tmp:
        tmp.append(_url)
urls = tmp

In [217]:
url

'https://home.cern/science/computing'

In [230]:
urls

['https://home.cern/science/physics/higgs-boson',
 'https://home.cern/science/physics/w-boson-sunshine-and-stardust',
 'https://home.cern/science/physics/z-boson',
 'https://home.cern/science/accelerators/large-hadron-collider',
 'https://home.cern/science/computing/birth-web',
 'https://home.cern/science/physics/antimatter',
 'https://home.cern/science',
 'https://home.cern/science/physics',
 'https://home.cern/science/physics/dark-matter',
 'https://home.cern/science/physics/early-universe',
 'https://home.cern/science/physics/standard-model',
 'https://home.cern/science/accelerators',
 'https://home.cern/science/accelerators/accelerator-complex',
 'https://home.cern/science/accelerators/antiproton-decelerator',
 'https://home.cern/science/accelerators/high-luminosity-lhc',
 'https://home.cern/science/engineering',
 'https://home.cern/science/engineering/accelerating-radiofrequency-cavities',
 'https://home.cern/science/engineering/pulling-together-superconducting-electromagnets',
 '

Reading the text in the `urls` from the website by using `UnstructedURLLoader`.

# DB establishment

## Download the embeddigns model

Download the embeddings model from the Hugging Face model hub.

In [225]:
if os.path.isfile(f"embedding_model/{embeddings_model_name}/config.json"):
    print("Model already exists.")
else:
    download_path = snapshot_download(
        repo_id=embeddings_model_name,
        local_dir = f"embedding_model/{embeddings_model_name}",
        local_dir_use_symlinks=False # If you want to use symlinks, set this to True
        )

Model already exists.


## Make retriever

In [233]:
import langchain_community.document_loaders

loader = langchain_community.document_loaders.UnstructuredURLLoader(
    urls=urls
)

In [234]:
docs = loader.load()

Error fetching or processing https://home.cernhttps://home.cern/user/login?destination=science/physics, exception: HTTPSConnectionPool(host='home.cernhttps', port=443): Max retries exceeded with url: /home.cern/user/login?destination=science/physics (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x448572f20>: Failed to resolve 'home.cernhttps' ([Errno 8] nodename nor servname provided, or not known)"))
Error fetching or processing https://home.cernhttps://home.web.cern.ch/science/physics/standard-model, exception: HTTPSConnectionPool(host='home.cernhttps', port=443): Max retries exceeded with url: /home.web.cern.ch/science/physics/standard-model (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x448567820>: Failed to resolve 'home.cernhttps' ([Errno 8] nodename nor servname provided, or not known)"))
Error fetching or processing https://home.cernhttps://home.cern/user/login?destination=science/accelerators, exception: HTTPSCon

In [ ]:
with open("docs.pkl", "wb") as f:
    pickle.dump(docs, f)

In [235]:
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 512,
    chunk_overlap = 32,
    length_function = len,
)

docs = text_splitter.split_documents(docs)
embeddings = HuggingFaceEmbeddings(
    model_name=embeddings_path
)

Created a chunk of size 550, which is longer than the specified 512
Created a chunk of size 565, which is longer than the specified 512
Created a chunk of size 771, which is longer than the specified 512
Created a chunk of size 552, which is longer than the specified 512
Created a chunk of size 586, which is longer than the specified 512
Created a chunk of size 617, which is longer than the specified 512
Created a chunk of size 584, which is longer than the specified 512
Created a chunk of size 826, which is longer than the specified 512
Created a chunk of size 805, which is longer than the specified 512
Created a chunk of size 575, which is longer than the specified 512
Created a chunk of size 926, which is longer than the specified 512
Created a chunk of size 999, which is longer than the specified 512
Created a chunk of size 713, which is longer than the specified 512
Created a chunk of size 1203, which is longer than the specified 512
Created a chunk of size 699, which is longer th

In [236]:
db = FAISS.from_documents(docs, embeddings)

In [237]:
retriever = db.as_retriever()

In [246]:
with open("retriever.pkl", "wb") as f:
    pickle.dump(retriever, f)

In [4]:
with open("retriever.pkl", "rb") as f:
    retriever = pickle.load(f)

# RAG chain

In [253]:
del LLM

Error in callback <bound method AutoreloadMagics.pre_run_cell of <IPython.extensions.autoreload.AutoreloadMagics object at 0x4486d5180>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 44857eec0, raw_cell="del LLM" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/masahirotojo/Documents/DH/rag.ipynb#Y161sZmlsZQ%3D%3D>,),kwargs {}:


TypeError: AutoreloadMagics.pre_run_cell() takes 1 positional argument but 2 were given

In [33]:
from helpers import LLM
llm = LLM()

Prompt Template Created
input_variables=['context', 'question', 'source'] input_types={} partial_variables={} template='\n        <|system|>\n        <s>\n        あなたは素粒子物理学のエキスパートです。\n        次の文脈を使って、最後の質問に答えてください。: {context}\n        そして、次のような係り受けで答える必要があります；\n            - 答えがわからない場合は、ただわからないと言い、答えを作ろうとしないでください。\n            - コードを表示するときはマークダウン・フォーマットを使用すること。\n            - 適切な文法と句読点を使用すること。\n            - ユーザーの助けになると思うのであれば、追加情報を加えてください。\n            - 段落ごとに改行してください。\n            - 完全な文章を使用してください。\n            - マークダウン書式を使用する。\n            - 専門的な用語は強調すること。\n            - 最後に改行して使用したsourceを示す。\n            - 答えの中で文章を繰り返さないこと。\n            - 日本語で回答すること。\n        </s>\n        <|user|>\n        <s>\n        {question}\n        </s>\n        <|source|>\n        <s>\n        {source}\n        </s>\n    '


In [41]:
question = "ATLAS検出器の構造は？"
response = llm.chat(question, retriever=retriever)

# Answer
ATLAS検出器は、LHCの4つの主要検出器の1つです。ATLAS検出器は、粒子衝突の詳細を捉えるために設計されています。検出器の構造は、以下の4つの主要部分で構成されています。
- **Inner Detector (ID)**: これは、粒子の軌跡を捉えるための最も内側の検出器です。IDは、シリコンの微細なピクセル検出器、シリコンのストリップ検出器、トランスション・レジストリー・トラッカー（TRT）で構成されています。
- **Calorimeters**: これらは、粒子のエネルギーを測定するための検出器です。ATLASのCalorimetersは、Electromagnetic Calorimeter（EMCal）とHadronic Calorimeter（HCal）で構成されています。
- **Muon Spectrometer**: これは、ミューオンの検出と測定を行うための検出器です。Muon Spectrometerは、ミューオン・トラッカーとミューオン・カロリメータで構成されています。
- **Trigger and Data Acquisition**: これは、検出器が捉えたデータを処理し、有用な情報を抽出するためのシステムです。

ATLAS検出器は、LHCの13TeVの衝突エネルギーで粒子を捉えるように設計されています。このエネルギーは、素粒子物理学の研究において、新しい粒子や相互作用の発見を可能にする高エネルギー領域を探検することを目的としています。
- **Source:** [1](https://home.cern/science/experiments/l3), [2](https://home.cern/science/experiments), [3](https://home.cern/science/engineering/restarting-lhc-why-13-tev), [4](https://home.cern/fr/science/accelerators)

In [42]:
print(response)

ATLAS検出器は、LHCの4つの主要検出器の1つです。ATLAS検出器は、粒子衝突の詳細を捉えるために設計されています。検出器の構造は、以下の4つの主要部分で構成されています。
- **Inner Detector (ID)**: これは、粒子の軌跡を捉えるための最も内側の検出器です。IDは、シリコンの微細なピクセル検出器、シリコンのストリップ検出器、トランスション・レジストリー・トラッカー（TRT）で構成されています。
- **Calorimeters**: これらは、粒子のエネルギーを測定するための検出器です。ATLASのCalorimetersは、Electromagnetic Calorimeter（EMCal）とHadronic Calorimeter（HCal）で構成されています。
- **Muon Spectrometer**: これは、ミューオンの検出と測定を行うための検出器です。Muon Spectrometerは、ミューオン・トラッカーとミューオン・カロリメータで構成されています。
- **Trigger and Data Acquisition**: これは、検出器が捉えたデータを処理し、有用な情報を抽出するためのシステムです。

ATLAS検出器は、LHCの13TeVの衝突エネルギーで粒子を捉えるように設計されています。このエネルギーは、素粒子物理学の研究において、新しい粒子や相互作用の発見を可能にする高エネルギー領域を探検することを目的としています。
- **Source:** [1](https://home.cern/science/experiments/l3), [2](https://home.cern/science/experiments), [3](https://home.cern/science/engineering/restarting-lhc-why-13-tev), [4](https://home.cern/fr/science/accelerators)
